In [1]:
import torch
from PIL import Image
from cleanlab import Datalab
import matplotlib.pyplot as plt
from pathlib import Path
from shutil import move
import pandas as pd


features = torch.load('/storage/results/labeled_6999//features.pt')
pred_probs = torch.load('/storage/results/labeled_6999/pred_probs.pt')
dataset = torch.load('/storage/results/labeled_6999/dataset.pt')
print(dataset)
imgs = dataset['image']

lab = Datalab(data=dataset, label_name="label", image_key="image")
lab.find_issues(features=features, pred_probs=pred_probs)

lab.report()


classes ={0:'no_watermark',1:'watermark'}



{'image': array(['/storage/data/labeled_6999/watermark/[ph]11625[ph]RBINSOPENUP_RBINS_BELGIUM_3204.jpg',
       '/storage/data/labeled_6999/watermark/[ph]431[ph]https___www_searchculture_gr_aggregator_edm_DigCollMIET_000119_694449.jpg',
       '/storage/data/labeled_6999/watermark/[ph]2024905[ph]photography_ProvidedCHO_NALIS_Foundation_F13515185290071.jpg',
       ...,
       '/storage/data/labeled_6999/no_watermark/[ph]92062[ph]BibliographicResource_1000126154170.jpg',
       '/storage/data/labeled_6999/no_watermark/[ph]92064[ph]bildarchivaustria_Preview_1234180.jpg',
       '/storage/data/labeled_6999/no_watermark/[ph]951[ph]Culturalia_96dda327_cf60_4988_aa38_5c6b5b81d9ce.jpg'],
      dtype='<U179'), 'label': array([1, 1, 1, ..., 0, 0, 0])}
Finding label issues ...
Finding outlier issues ...
Fitting OOD estimator based on provided features ...
Finding near_duplicate issues ...
Finding non_iid issues ...
Finding dark, light, low_information, odd_aspect_ratio, odd_size, grayscale, blur

  0%|          | 0/6999 [00:00<?, ?it/s]

Error in checking for image issues: 'str' object has no attribute 'size'

Audit complete. 1171 issues found in the dataset.
Here is a summary of the different kinds of issues found in the data:

    issue_type  num_issues
         label         887
       outlier         154
near_duplicate         129
       non_iid           1

Dataset Information: num_examples: 6999, num_classes: 2


----------------------- label issues -----------------------

About this issue:
	Examples whose given label is estimated to be potentially incorrect
    (e.g. due to annotation error) are flagged as having label issues.
    

Number of examples with this issue: 887
Overall dataset quality in terms of this issue: 0.8733

Examples representing most severe instances of this issue:
      is_label_issue   label_score  given_label  predicted_label
6955            True  4.017595e-20            0                1
4843            True  1.925082e-14            1                0
3536            True  5.977785e-14 

In [ ]:
issues = lab.get_issues("outlier")
#issues = issues.sort_values(by='outlier_score', ascending = True)
issues = issues.loc[issues['is_outlier_issue'] == True]

#issues = issues.head(2)
print(issues)


In [ ]:
for path in imgs[issues.index]:
    print(path)
    img = Image.open(path)
    
    fig,ax = plt.subplots(1,1)
    ax.imshow(img)
    ax.axis("off")
    plt.show()
    print(2*"\n")

In [ ]:
issues = lab.get_issues("near_duplicate")

issues = issues.loc[issues['is_near_duplicate_issue'] == True]
print(issues)


In [ ]:
issues.index

In [ ]:
for i, row in issues.iterrows():
    #print(row['near_duplicate_sets'])
    path = imgs[i]
    path_dup = [imgs[l] for l in row['near_duplicate_sets']][:3]

    print(path)


    img = Image.open(path)

    fig,ax = plt.subplots(1,len(path_dup)+1)
    ax[0].imshow(img)
    ax[0].axis("off")

    for i,path in enumerate(path_dup):
        #print(i)
        img = Image.open(path)
        ax[i+1].imshow(img)
        ax[i+1].axis("off")
    plt.show()
    print(2*"\n")



In [ ]:
for path in imgs[issues.index]:
    print(path)
    img = Image.open(path)
    
    fig,ax = plt.subplots(1,1)
    ax.imshow(img)
    ax.axis("off")
    plt.show()
    print(2*"\n")

In [2]:


metadata_path = '/storage/data/labeled_6999.csv'

metadata_df = pd.read_csv(metadata_path)

issues = lab.get_issues("label")
issues = issues.loc[issues['is_label_issue'] == True]

issues['predicted_label'] = issues['predicted_label'].apply(lambda x: classes[x])
issues['given_label'] = issues['given_label'].apply(lambda x: classes[x])
issues = issues.drop(columns = ['is_label_issue','label_score'])

issues['id'] = [Path(path).with_suffix('').name.replace('[ph]','/') for path in imgs[issues.index]]

issues = issues.merge(metadata_df, left_on='id', right_on='id')
issues = issues.drop(columns = ['category'])

print(issues)
print(issues.shape)

issues.to_csv('/storage/results/labeled_6999/images_to_review.csv',index = False)


      given_label predicted_label                                        id  \
0       watermark    no_watermark                              /92085/11961   
1       watermark    no_watermark       /2022719/arpadweb_bib_ADGD160967448   
2       watermark    no_watermark              /91674/GSM_delobjekt_1320365   
3       watermark    no_watermark                             /15501/011097   
4       watermark    no_watermark                             /15501/002344   
..            ...             ...                                       ...   
882  no_watermark       watermark        /9200518/ark__12148_btv1b53114069p   
883  no_watermark       watermark         /9200518/ark__12148_btv1b9052314c   
884  no_watermark       watermark                         /9200579/kg56dtu6   
885  no_watermark       watermark                         /9200579/zcdskcsh   
886  no_watermark       watermark  /92064/bildarchivaustria_Preview_1234180   

                                                   

In [16]:
import pandas as pd


metadata_path = '/storage/data/labeled_6999.csv'

metadata_df = pd.read_csv(metadata_path)
metadata_df['dataset'] = metadata_df['id'].apply(lambda x: x.split('/')[1])

watermarks_df = metadata_df.loc[metadata_df['category'] == 'watermark']
watermark_datasets = watermarks_df['dataset'].unique()
len(watermark_datasets)

154

In [19]:
for dataset_id in watermark_datasets:
    df = watermarks_df.loc[watermarks_df['dataset'] == dataset_id]
    print(df.shape)

(22, 5)
(52, 5)
(65, 5)
(50, 5)
(51, 5)
(37, 5)
(51, 5)
(58, 5)
(49, 5)
(30, 5)
(45, 5)
(50, 5)
(43, 5)
(50, 5)
(53, 5)
(61, 5)
(51, 5)
(31, 5)
(47, 5)
(52, 5)
(51, 5)
(51, 5)
(52, 5)
(53, 5)
(49, 5)
(49, 5)
(60, 5)
(51, 5)
(14, 5)
(14, 5)
(24, 5)
(29, 5)
(13, 5)
(1, 5)
(1, 5)
(42, 5)
(7, 5)
(10, 5)
(3, 5)
(1, 5)
(5, 5)
(1, 5)
(9, 5)
(2, 5)
(148, 5)
(6, 5)
(1, 5)
(6, 5)
(2, 5)
(1, 5)
(141, 5)
(7, 5)
(40, 5)
(1, 5)
(1, 5)
(1, 5)
(1, 5)
(3, 5)
(2, 5)
(1, 5)
(2, 5)
(10, 5)
(1, 5)
(2, 5)
(8, 5)
(13, 5)
(9, 5)
(3, 5)
(1, 5)
(2, 5)
(4, 5)
(8, 5)
(1, 5)
(2, 5)
(2, 5)
(1, 5)
(130, 5)
(119, 5)
(1, 5)
(2, 5)
(6, 5)
(10, 5)
(8, 5)
(13, 5)
(1, 5)
(1, 5)
(2, 5)
(1, 5)
(2, 5)
(8, 5)
(1, 5)
(2, 5)
(10, 5)
(1, 5)
(1, 5)
(1, 5)
(8, 5)
(2, 5)
(1, 5)
(1, 5)
(8, 5)
(1, 5)
(6, 5)
(1, 5)
(1, 5)
(1, 5)
(1, 5)
(3, 5)
(2, 5)
(3, 5)
(5, 5)
(10, 5)
(5, 5)
(2, 5)
(1, 5)
(1, 5)
(1, 5)
(1, 5)
(1, 5)
(50, 5)
(2, 5)
(3, 5)
(1, 5)
(145, 5)
(4, 5)
(2, 5)
(1, 5)
(7, 5)
(11, 5)
(8, 5)
(4, 5)
(1, 5)
(166, 5)
(117, 5)
(55,

In [12]:
len()

154

In [7]:
metadata_df

,id,category,uri,image_url,dataset
0,/916106/vbg_object_VM02937_b,watermark,https://www.europeana.eu/en/item/916106/vbg_ob...,http://mm.dimu.org/image/012uL2QqcFRs?dimensio...,916106
1,/11625/RBINSOPENUP_RBINS_BELGIUM_3204,watermark,https://www.europeana.eu/en/item/11625/RBINSOP...,http://projects.biodiversity.be/openup/rbins/3...,11625
2,/91625/nomu_object_NM0154251,watermark,https://www.europeana.eu/en/item/91625/nomu_ob...,http://mm.dimu.org/image/022s84LRRAoh?dimensio...,91625
3,/431/https___www_searchculture_gr_aggregator_e...,watermark,https://www.europeana.eu/en/item/431/https___w...,https://www.searchculture.gr/aggregator/thumbn...,431
4,/91642/SMVK_OM_delobjekt_118274,watermark,https://www.europeana.eu/en/item/91642/SMVK_OM...,http://collections.smvk.se/carlotta-om/web/ima...,91642
...,...,...,...,...,...
6994,/946/https___www_searchculture_gr_aggregator_e...,no_watermark,https://www.europeana.eu/en/item/946/https___w...,https://www.searchculture.gr/aggregator/thumbn...,946
6995,/951/Culturalia_43e7ad75_ea7c_4013_934b_984c6d...,no_watermark,https://www.europeana.eu/en/item/951/Culturali...,NaN,951
6996,/951/Culturalia_96dda327_cf60_4988_aa38_5c6b5b...,no_watermark,https://www.europeana.eu/en/item/951/Culturali...,NaN,951
6997,/951/Culturalia_b7b2ee8d_8ea2_4e07_82ec_70c53b...,no_watermark,https://www.europeana.eu/en/item/951/Culturali...,NaN,951


In [18]:
issues

,is_label_issue,label_score,given_label,predicted_label
9,True,1.426999e-02,watermark,no_watermark
47,True,9.134066e-03,watermark,no_watermark
75,True,1.725217e-01,watermark,no_watermark
87,True,2.028626e-02,watermark,no_watermark
127,True,3.516130e-02,watermark,no_watermark
...,...,...,...,...
4222,True,2.445217e-01,no_watermark,watermark
4235,True,6.942308e-02,no_watermark,watermark
4239,True,6.960711e-04,no_watermark,watermark
4258,True,3.968944e-02,no_watermark,watermark


In [ ]:
issues = lab.get_issues("label")
issues = issues.loc[issues['is_label_issue'] == True]

explore = 'watermark'

inv_class = {v:k for k,v in classes.items()}
issues = issues.loc[issues['predicted_label'] == inv_class[explore]]

issues = issues.tail(36)
print(issues.shape)

N = 50

num = 0

for i,row in issues.iterrows():
    try:
        img = Image.open(imgs[i])
        num += 1
        
    except:
        continue

    if num > N:
        break

    print(imgs[i])
    print(f"gt: {classes[row['given_label']]}, predicted: {classes[row['predicted_label']]}")
    fig,ax = plt.subplots(1,1,figsize = (10,10))
    ax.imshow(img)
    ax.axis('off')
    plt.show()
    print(2*"\n")

In [ ]:


dest_path = Path('/storage/data/labeled_4312/watermark/')

with open('/storage/results/iter_6/to_watermarks.txt','r') as f:
    path_list = f.readlines()

for path in path_list:
    path = Path(path.strip())
    if not path.is_file():
        continue
    move(str(path),str(dest_path.joinpath(path.name)))
    

    

    

In [ ]:
from torch.utils.data import DataLoader, TensorDataset, Subset
import torch
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import StratifiedKFold
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import math
import time
import multiprocessing

from cleanlab import Datalab

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Method for computing out-of-sample embeddings
def compute_embeddings(model, testloader):
    embeddings_list = []

    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data[0].to(device), data[1].to(device)

            embeddings = model.embeddings(images)
            embeddings_list.append(embeddings.cpu())

    return torch.vstack(embeddings_list)


# Method for computing out-of-sample predicted probabilities
def compute_pred_probs(model, testloader):
    pred_probs_list = []

    with torch.no_grad():
        for data in tqdm(testloader):
            images, labels = data[0].to(device), data[1].to(device)

            outputs = model(images)
            pred_probs_list.append(outputs.cpu())

    return torch.vstack(pred_probs_list)

In [ ]:
import sys
sys.path.append('../machine-learning')

from train import Classifier, TrainingDataset, my_collate

import fire
from pathlib import Path
import torchvision
from torchvision import transforms
import numpy as np
from sklearn import preprocessing
import torch
import torchmetrics
import pytorch_lightning as pl
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
import json

from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from torch.utils.data import Dataset, DataLoader
from PIL import Image

from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

class Classifier(pl.LightningModule): 
  
    def __init__(self, **kwargs):
        super().__init__()
        self.save_hyperparameters()

        output_dim = kwargs.get('output_dim')
        learning_rate = kwargs.get('learning_rate')
        self.threshold = kwargs.get('threshold')

        self.model = torchvision.models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, output_dim)
        self.embedding = nn.Sequential(*list(self.model.children())[:-1])

        #self.output = nn.Linear(self.model.fc.out_features, output_dim)
        
        self.sm = nn.Softmax(dim=1)
        self.accuracy = torchmetrics.Accuracy(task='binary')

    def forward(self, x):
        out = self.model(x)
        #out = self.output(x)
        out = self.sm(out)
        return out

    def embeddings(self, x):
        out = self.embedding(x)
        return out

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        y_hat = torch.where(y_hat>self.threshold,1,0).int()
        self.accuracy(y_hat, y.int())
        self.log('valid_acc_step', self.accuracy)
        self.log('valid_loss', loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        
        loss = F.cross_entropy(y_hat, y)
        y_hat = torch.where(y_hat>self.threshold,1,0).int()
        self.accuracy(y_hat, y.int())
        self.log('test_acc_step', self.accuracy)
        self.log('test_loss', loss)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

In [ ]:


data_dir = '/output/data/labeled_4312'
saving_dir = '/output/results/iter_7'

max_epochs = 1
sample = 0.6
#test_size = 0.2
batch_size = 16
learning_rate = 1e-4
threshold = 0.5
num_workers = 1
patience = 5
K = 3

data_dir = Path(data_dir)
saving_dir = Path(saving_dir)
saving_dir.mkdir(exist_ok = True, parents=True)

train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomApply([transforms.Grayscale(num_output_channels=3)], p=0.25),
    #transforms.RandomRotation(degrees=(0, 45)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])



imgs = np.array([str(p) for p in data_dir.rglob("*/*")])
labels = np.array([Path(p).parent.name for p in imgs])

n = int(imgs.shape[0]*sample)
imgs = imgs[:n]
labels = labels[:n]


le = preprocessing.LabelEncoder()
_labels = le.fit_transform(labels)
classes = le.classes_
labels = F.one_hot(torch.from_numpy(_labels)).float()

print(len(classes))


# Create k splits of the dataset

kfold = StratifiedKFold(n_splits=K, shuffle=True, random_state=0)
splits = kfold.split(imgs, _labels)

train_id_list, test_id_list = [], []

for fold, (train_ids, test_ids) in enumerate(splits):
    train_id_list.append(train_ids)
    test_id_list.append(test_ids)


pred_probs_list, embeddings_list = [], []
embeddings_model = None

for i in range(K):
    print(f"\nTraining on fold: {i+1} ...")

    # Create train and test sets and corresponding dataloaders

    X_train_val = imgs[train_id_list[i]]
    y_train_val = labels[train_id_list[i]]

    # Create validation split

    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=1.0/K)

    X_test = imgs[test_id_list[i]]
    y_test = labels[test_id_list[i]]

    train_dataset = TrainingDataset(X_train,y_train,transform = train_transform)
    val_dataset = TrainingDataset(X_val,y_val,transform = test_transform)
    test_dataset = TrainingDataset(X_test,y_test,transform = test_transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size,collate_fn=my_collate,num_workers=num_workers)
    val_loader = DataLoader(val_dataset, batch_size=batch_size,collate_fn=my_collate,num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=batch_size,collate_fn=my_collate,num_workers=num_workers)

    model = Classifier(
        output_dim = len(classes), 
        learning_rate = learning_rate,
        threshold = threshold
    )

    callbacks = [EarlyStopping(monitor="valid_loss",patience=patience, verbose = True)]

    trainer = pl.Trainer(
        #gpus=1,
        accelerator="auto",
        max_epochs = max_epochs,
        log_every_n_steps=100,
        callbacks = callbacks
    )

    trainer.fit(model, train_loader, val_loader)
    trainer.test(dataloaders=test_loader)

    model = model.to(device)

    # Compute out-of-sample embeddings
    print("Computing feature embeddings ...")
    fold_embeddings = compute_embeddings(model, test_loader)
    embeddings_list.append(fold_embeddings)

    print("Computing predicted probabilities ...")
    # Compute out-of-sample predicted probabilities
    fold_pred_probs = compute_pred_probs(model, test_loader)
    pred_probs_list.append(fold_pred_probs)

print("Finished Training")


# Combine embeddings and predicted probabilities from each fold
features = torch.vstack(embeddings_list).numpy()
features = np.squeeze(features)

logits = torch.vstack(pred_probs_list)
pred_probs = logits.numpy()

indices = np.hstack(test_id_list)

imgs = imgs[indices]
_labels = _labels[indices]
dataset = {'image':imgs,'label':_labels}

saving_dir

torch.save(features, '/output/results/features.pt')
torch.save(dataset, '/output/results/dataset.pt')
torch.save(pred_probs, '/output/results/pred_probs.pt')


In [ ]:
classes

